In [2]:
import lwa_lib
import matplotlib.pyplot as plt
from file_management_lib import get_paths
import numpy as np

from matplotlib import pyplot as plt, ticker as mticker
from scipy import integrate


In [3]:
plt.style.library['tableau-colorblind10']


RcParams({'axes.prop_cycle': cycler('color', ['#006BA4', '#FF800E', '#ABABAB', '#595959', '#5F9ED1', '#C85200', '#898989', '#A2C8EC', '#FFBC79', '#CFCFCF']),
          'patch.facecolor': '#006BA4'})

In [ ]:
multiplier = 1.5
fontsize = 26
plt.style.use('default')
plt.rcParams['lines.linewidth'] = 3 * multiplier
plt.rcParams['axes.labelsize'] = fontsize * multiplier

plt.rcParams['legend.fontsize'] = fontsize * multiplier
plt.rcParams['legend.loc'] = 'upper right'
plt.rcParams['xtick.labelsize'] = fontsize * multiplier
plt.rcParams['ytick.labelsize'] = (fontsize)* multiplier
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['xtick.minor.size'] = 4 * multiplier
plt.rcParams['xtick.minor.width'] = 1.5*multiplier
plt.rcParams['xtick.major.width'] = 1.5* multiplier
plt.rcParams['ytick.minor.width'] = 1.5 * multiplier
plt.rcParams['ytick.major.width'] = 1.5 * multiplier
plt.rcParams['xtick.major.size'] = 7 * multiplier
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.size'] = 4 * multiplier

plt.rcParams['ytick.major.size'] = 7 * multiplier

plt.rcParams["legend.framealpha"] = 1 * multiplier




path_high_finesse = r"C:\Users\au617810\OneDrive - Aarhus universitet\Videnskabelig assistent\ECOC2025\Weird_data\Weird_data\HighFinesse"
path_CSH_200 = r"C:\Users\au617810\OneDrive - Aarhus universitet\Videnskabelig assistent\ECOC2025\Weird_data\Weird_data\FN_200_counts"
path_CSH_50 = r"C:\Users\au617810\OneDrive - Aarhus universitet\Videnskabelig assistent\ECOC2025\Weird_data\Weird_data\FN_50_counts"


# comparison_dir = r".\Data\Frequency_noise\HighFinesse\28-1"
# comparison_paths = get_paths(comparison_dir)

# cur_dir = [path for path in comparison_paths if '-17' in path][0]


def get_comparison_directory(cur_dir):
    csh, hf = get_comparison_paths(cur_dir)
    lw_csh = []
    lw_hf = []
    
    for path_csh, path_hf in zip(csh,hf):
        floor_csh, floor_hf = compare(path_csh,path_hf)
        lw_csh.append(floor_csh*np.pi)
        lw_hf.append(floor_hf*np.pi)
    return lw_csh, lw_hf

def get_noise_floor(freqs,ps,csh=True):
    if csh:
        condition = (freqs > 9e5) & (freqs < 1e6)
    if not csh:
        condition = (freqs > 5e6) & (freqs < 6e6) 
    new_freqs = freqs[condition]
    new_ps = ps[condition]
    floor = np.mean(new_ps)
    return floor
    

def get_comparison_paths(directory):
    csh = []
    hf = []
    for path in get_paths(directory):
        if 'sub' in path:
            csh.append(path)
        if 'PSD' in path:
            hf.append(path)
    return np.array(csh), np.array(hf)



def compare(path_csh,path_hf):
    lwa = lwa_lib.LWA(path_hf)
    fs_hf = lwa.freqs
    ps_hf = lwa.powers
    fs_csh, ps_csh, lw_csh = get_coherent_dsh2(path_csh)
    floor_csh = get_noise_floor(fs_csh,ps_csh,csh=True)
    floor_hf = get_noise_floor(fs_hf,ps_hf,csh=False)
    
    #plt.figure(figsize= [15,8])
    #plt.loglog(fs_hf,ps_hf,label='HighFinesse')
    #plt.loglog(fs_csh,ps_csh,label='CSH')
    #plt.ylim([1e0,1e8])
    return floor_csh, floor_hf
    

def get_coherent_dsh2(path):
    data = np.loadtxt(path,skiprows=1)
    freqs = data[:,0]
    fn = data[:,1]
    lw = min(fn)
    return freqs, fn, lw

def plot_new2():
    path3 = r".\Data\Frequency_noise\good_slope_216uW_355uW_fb_100counts_100hz_bw.txt"
    path5 = r".\Data\Frequency_noise\good_slope_no_fb_200_counts_100hz_bw.txt"
    

    no_fb = get_paths(path_CSH_200)
    high_fb = get_paths(path_CSH_50)

    
    print(high_fb2[1])
    
    freqs1, ps1, lw1 = get_coherent_dsh2(path3)
    freqs2, ps2, lw2 = get_coherent_dsh2(path5)
    freqs3, ps3, lw3 = get_coherent_dsh2(no_fb2[2])
    freqs4, ps4, lw4 = get_coherent_dsh2(high_fb2[6])
    print('Linewidth:')
    print(lw4)
    lwa = lwa_lib.LWA(no_fb2[6])
    lwa2 = lwa_lib.LWA(high_fb2[1])
    
    
    #print(lw2)
    lw = 4
    fig, ax = plt.subplots(figsize=[10 * multiplier,8 * multiplier])
    color1 = 'blue'

    beta_freqs = np.logspace(2,7,num=100)
    beta_line = 8*np.log(2)*beta_freqs/np.pi**2

    ax.plot(lwa.freqs,lwa.powers,linewidth=lw,color='#898989', label='FNA: No feedback')
    ax.plot(lwa2.freqs,lwa2.powers,linewidth=lw,color='#595959', label='FNA: -11 dB misaligned')
    ax.plot(freqs3,ps3,linewidth=lw,color='#5F9ED1', label='CSH: No feedback')
    ax.plot(freqs4,ps4,linewidth=lw,color='#006BA4', label='CSH: -11 dB misaligned')
    ax.plot(beta_freqs,beta_line,linestyle='--',color='#FF800E', alpha = .6, label = 'Beta separation line',zorder=10)
    
    
    ax.minorticks_on()
    ax.yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))
    
    #ax.axhline(4e5)
    #ax.axhline(50)
    ax.set(xlim=[1e2,1e7],
           ylim=[1e0,2e10],
           xscale='log',
           yscale='log',
           yticks=[1,1e2,1e4,1e6,1e8,1e10],
           xlabel='Fourier frequency [Hz]',
           ylabel='FN PSD [Hz$^2$/Hz]')
    ax.grid()
    
    
    
    condition1 = (freqs4 > 3e5) & (freqs4 < 1e6)
    condition2 = (freqs2 > 3e6) & (freqs2 < 3.5e6)
    ps1_intrinsic = ps4[condition1]
    ps2_intrinsic = ps2[condition2]
    
    print(np.mean(ps1_intrinsic))
    print(np.mean(ps2_intrinsic))
    #ax.axhline(100)
    ax.legend(fontsize=27,loc=[.50,.69],framealpha=0.8,ncols=1)
    
    nfs, nps,integral1,eff_lw = beta_sep_line(freqs4,ps4,4.5e3)
    _,_,_,eff_lw2 = beta_sep_line(freqs3,ps3,9.2e5)

    plt.savefig(r"O:\Tech_Photonics\Projects\Narrow Linewidth\MFB Chips\Chip 3 Feedback measurements\SPIE Prague 2025\Figures\CSH_FN_PSD.svg",bbox_inches='tight')
    print(eff_lw)

        
def beta_sep_line(freqs,ps,cutoff):
    condition = (freqs < cutoff) & (freqs > 0)
    new_freqs = freqs[condition]
    new_ps = ps[condition]
    integral = integrate.cumtrapz(new_ps,new_freqs,initial=0)
    A = integral[-1]
    return new_freqs, new_ps, integral, np.sqrt(8*np.log(2)*A)

plot_new2()

def get_data():
    result = []
    for path in comparison_paths:
        if 'dB' in path:
            lw_csh, lw_hf = get_comparison_directory(path)
            feedback = int(path.split('\\')[-1][0:3])
            result.append([feedback,lw_csh,lw_hf])
    
    feedbacks, lws_csh, lws_hf = zip(*result)
    return feedbacks, lws_csh, lws_hf

feedbacks,lws_csh,lws_hf = get_data()




In [5]:
lws_csh_avg = np.array([np.average(lws_csh[i]) for i in range(8)])
lws_csh_std = np.array([np.std(lws_csh[i]) for i in range(8)])

feedbacks = np.array(feedbacks)


lws_hf_avg = np.array([np.average(lws_hf[i]) for i in range(8)])
lws_hf_std = np.array([np.std(lws_hf[i]) for i in range(8)])

np.savetxt(r'.\Data\Frequency_noise\csh_data.txt', (lws_csh_avg,lws_csh_std,feedbacks,lws_hf_avg,lws_hf_std), header='Average CSH Linewidth, Standard deviation CSH linewidth, Feedback ratio, Average LWA Linewidth, Standard deviation LWA linewidth' )
